In [1]:
import lightgbm
import pandas as pd
import numpy as np

In [55]:
init_record = pd.read_csv("../../Data/LWB_Monthly_data.csv")


In [56]:
init_record['_key_occurreddate_month']=init_record['_key_occurreddate_month'].astype("datetime64[ns]")
init_record = init_record.set_index(['_key_client_id', '_key_occurreddate_month'])

# Try reduce columns

In [57]:
selected = pd.concat([init_record.filter(regex=r'allrows', axis=1), init_record["response_variable"]], axis = 1)
#selected = init_record.drop(init_record.filter(regex=r'allrows', axis=1),axis = 1)
selected

allrows_client_didexistprecirts  \
_key_client_id                       _key_occurreddate_month                                    
0000D172-EA88-432F-8235-9FAA00D29072 2019-04-01                                             1   
                                     2019-05-01                                             1   
                                     2019-06-01                                             1   
                                     2019-07-01                                             1   
                                     2019-08-01                                             1   
...                                                                                       ...   
FFFF7C4D-F6DD-46D0-A9C0-AE3C2DC51132 2019-09-01                                             0   
                                     2019-10-01                                             0   
                                     2019-11-01                                             0   
                                     2019-12-01                                             0   
                                     2020-01-01                                             0   

                                                              allrows_client_isactive  \
_key_client_id                       _key_occurreddate_month                            
0000D172-EA88-432F-8235-9FAA00D29072 2019-04-01                                     1   
                                     2019-05-01                                     1   
                                     2019-06-01                                     1   
                                     2019-07-01                                     1   
                                     2019-08-01                                     1   
...                                                                               ...   
FFFF7C4D-F6DD-46D0-A9C0-AE3C2DC51132 2019-09-01                                     0   
                                     2019-10-01                                     0   
                                     2019-11-01                                     0   
                                     2019-12-01                                     0   
                                     2020-01-01                                     0   

                                                              allrows_client_statusreason  \
_key_client_id                       _key_occurreddate_month                                
0000D172-EA88-432F-8235-9FAA00D29072 2019-04-01                                       0.0   
                                     2019-05-01                                       0.0   
                                     2019-06-01                                       0.0   
                                     2019-07-01                                       0.0   
                                     2019-08-01                                       0.0   
...                                                                                   ...   
FFFF7C4D-F6DD-46D0-A9C0-AE3C2DC51132 2019-09-01                                       6.0   
                                     2019-10-01                                       6.0   
                                     2019-11-01                                       6.0   
                                     2019-12-01                                       6.0   
                                     2020-01-01                                       6.0   

                                                              allrows_client_gender  \
_key_client_id                       _key_occurreddate_month                          
0000D172-EA88-432F-8235-9FAA00D29072 2019-04-01                                   1   
                                     2019-05-01                                   1   
                                     2019-06-01                                   1 

In [58]:
def construct_feature_label(data):
    y = []
    clientList = list(set(data.index.get_level_values(0)))
    data['response_variable'] = data.groupby(level=0)['response_variable'].shift(-1)  
    data = data.groupby(level=0, as_index=False).apply(lambda group: group.iloc[:-1])
    #print(data.loc["0000D172-EA88-432F-8235-9FAA00D29072"]['response_variable'])
    y = data['response_variable']
    x = data.drop('response_variable',axis='columns')
    return x, y

feature, label = construct_feature_label(selected)


# PCA

In [59]:
from sklearn import decomposition


In [60]:
#pca = decomposition.PCA(n_components=0.9)
#feature_pca = pca.fit_transform(feature)

# Model

In [61]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score, confusion_matrix, roc_curve,accuracy_score


In [62]:
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)
model.fit(feature)

IsolationForest(contamination=0.1, n_estimators=50)

In [63]:
y_hat = model.predict(feature)

In [64]:
y_hat = np.where(y_hat == -1, 1, 0) 

In [65]:
f1_score(label, y_hat)

0.3857910385478066

In [66]:
confusion_matrix(label,y_hat)

array([[94433,  7290],
       [ 4485,  3698]])

In [67]:
accuracy_score(label,y_hat)

0.8928629920113551

In [68]:
# Try rollout Data